In [1]:
from bienc_model import biencoder_model
from data_mt5gen import data_mt5gen
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import math
import torch
dl = data_mt5gen('train')

d:\install\Anaconda\envs\py3713\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_batch(sample):
    query = [each[0] for each in sample]
    context = [each[1] for each in sample]
    label = [each[2] for each in sample]
    qmax = max([len(each) for each in query])
    cmax = max([max([len(c) for c in q]) for q in context])
    q = torch.zeros((len(query), qmax), dtype=torch.int64)
    c = torch.zeros((len(context), len(context[0]), cmax), dtype=torch.int64)
    l = torch.tensor(label)
    for i in range(len(query)):
        for j in range(len(query[i])):
            q[i, j] = query[i][j]
        for j in range(len(context[i])):
            for k in range(len(context[i][j])):
                c[i, j ,k] = context[i][j][k]
    return q, c, l

model = biencoder_model()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train_dataloader = DataLoader(dl, batch_size=2, collate_fn=get_batch, shuffle=True)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.predictions.bias', 'cls.pred

In [3]:
model = model.to(device)
epochs = 900
PATH = './roberta_retriver'

class custom_loss(torch.nn.Module):
    def __init__(self):
        super(custom_loss, self).__init__()
        
    def forward(self, output, target):
        temp = 0
        for i in output:
            temp += math.exp(i)
            #print(math.exp(i))
        #print(math.exp(output[target]) ,temp)
        return - torch.log(torch.tensor(math.exp(output[target]) / temp).requires_grad_())

#criterion = nn.MSELoss()
criterion = torch.nn.CrossEntropyLoss()
#criterion = custom_loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00003, weight_decay=0.0001)
optimizer.zero_grad()
min_loss = 0
for epoch in range(epochs):
    
#     if (epoch) % 4 == 0:
#         for param in model.module.parameters():
#             param.requires_grad = True
#     else:      # 加強對query的訓練
#         for param in model.module.parameters():
#             param.requires_grad = False
#         for param in model.module.context_encoder.parameters():
#             param.requires_grad = True
    
    acc_loss_n = 1  # greadient acc times
    running_loss = 0.0
    epoch_loss = 0.0  # 紀錄用
    epoch_bar = tqdm(train_dataloader)
    epoch_bar.set_description("epoch: %d , loss: %f , min: %f, batch: " %(epoch + 1, running_loss, min_loss))
    
    for  i, data in enumerate(epoch_bar):
        query, context, label = data
        outputs = model(query.to(device), context.to(device))#, label.to(device))
        #print(outputs)
        #print(same_r, diff_r)
        loss = criterion(outputs, label.to(device))
        #loss = outputs
        #loss = loss / acc_loss_n
        #print(loss)
        loss.backward()
        
        running_loss += loss.item() * acc_loss_n
        if ((i + 1) % acc_loss_n == 0):
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss = running_loss / (i + 1)
            epoch_bar.set_description("epoch: %d , loss: %f , min: %f, batch: " %(epoch + 1, epoch_loss, min_loss))
        #break  # test
    
    if (epoch_loss <= min_loss) or (epoch == 0):
        min_loss = epoch_loss
        epoch_bar.set_description("epoch: %d , loss: %f , min: %f, batch: " %(epoch + 1, epoch_loss, min_loss))
        torch.save({'model': model.state_dict()}, 'roberta_retriver_min.pth.tar')
    
    if (epoch % 1) == 0:
        torch.save({'model': model.state_dict()}, "roberta_retriver_" + str(epoch + 1) +".pth.tar")
        
    epoch_bar.close()
    #break  # test
#     if (epoch % 1) == 0:
#         torch.save({'model': model.state_dict()}, 'roberta_retriver.pth.tar')
        #torch.save(model, PATH + "_" + str(epoch + 1) + ".pth")
print('Finished Training')
# ==================================================================
# 儲存模型
#torch.save({'model': model.state_dict()}, 'roberta_retriver.pth.tar')
#torch.save(model, PATH + "_" + str(epoch + 1) + ".pth")

epoch: 1 , loss: 0.000000 , min: 0.000000, batch: :   0%|          | 0/100000 [00:02<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 92.00 MiB (GPU 0; 11.00 GiB total capacity; 9.79 GiB already allocated; 0 bytes free; 10.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF